# Assignment 2: Noisy channel models and the Viterbi algorithm

## Due: by 11:59pm, Sunday 11/03

*Note: late submissions are accepted up to 5 days after the deadline for this assignment. You will receive 4 free late days over the course; once you exhaust those late days, each (partial) day that a submission is late will cap your maximum grade for the assignment by 5%. If you submit this assignment more than 5 days late, you will receive no credit.*

**Before submitting**, please click the *Kernel* menu at the top of JupyterLab and select the *Restart Kernel and Run All Cells...* button, then click the red *Restart* button on the box that pops up. This ensures that your code will run on its own, without relying on information that you may have added and then removed when developing it.

**To submit** your assignment, first make sure you have saved it, by pressing *Ctrl + S* or *Cmd + S* or clicking on the disk icon at the top of the notebook. Then enter the command `submit_assignment 2` in the *Terminal*. You can submit as many times as you like, without penalty. If you'd like to discuss something from your assignment with us (e.g. in office hours), please submit it first, so that we can easily see what you've been working on.

When you submit, a message will appear asking you if you want to mark the submission as FINAL. **You need to mark the submission as FINAL in order to have it graded.** After marking your submission as FINAL, you will no longer be able to re-submit. This is so that we can accurately track the date of your final submission with respect to the deadline.

## Please answer these questions prior to submitting

<span style="color: red;">**Did you consult anyone other than the instructor, or any resources other than those listed on Canvas, for this assignment? If so, please list them below.**</span>

> It is fine to consult classmates and external resources, as long as the work you submit is your own. I would just like to know who you consulted, or what other resources you might have found helpful.

I discussed with yanlu about the part 2.


<span style="color: red;">**For Part 3, have you chosen to complete option A (real-world effects) or option B (applications to language-based research)?**</span>

> Remember, you can only choose each option *twice* over the four assignments.

option A

<span style="color: red;">**For which part of the assignment would you like to receive double points?**</span>

> You can choose Part 1, 2, or 3, and you can choose the same or different parts across assignments with no restrictions.

Part 2

# Part 1: Autocorrect with Noisy Channel modeling [25 pts]

In this part of the assignment, you will use a provided codebase to explore the influence of various components of the noisy channel spelling model. *Note: the codebase is written for exploratory purposes, not for efficiency. It would not be a good tool to use for any large-scale practical tasks!*

**You must run the code cells below in order to be able to use the autocorrect tools in the notebook.**

In [2]:
import arpa
from util import run_srilm, ErrorModel, CandidateGenerator, SpellCorrector
import math
import pandas as pd 

simple_candidate_gen = CandidateGenerator(1, "inputs/words.txt")
uniform_error_model = ErrorModel("inputs/errors_uniform.txt")
uniform_language_model = arpa.loadf("models/uniform.lm")[0]
simple_corrector = SpellCorrector(simple_candidate_gen, uniform_error_model, uniform_language_model, noerror_rate=0)

## The spelling corrector

The codebase for this assignment provides you with a `SpellCorrector` object, which combines a candidate generator, an error model, and a language model, in order to perform spelling correction.

One of the primary methods of the spelling corrector is `SpellCorrector.correct_word()`, which evaluates candidates for correcting a single word in a sentence. The method takes two arguments: a string representing a sentence, and an integer representing the index of the word in the sentence that is to be corrected (as in all things Python, the first word is indexed by the number 0). It returns a DataFrame containing information about each candidate, including its scores (log-probabilities) on the error model and language model separately and together. The candidates (rows) in the DataFrame are sorted so that the best candidate appears at the top.

The code cell below demonstrates.

In [3]:
simple_corrector.correct_word("hlelo world", 0)

,target,candidate,distance,position,sentence,error_score,lm_score,score
0,hlelo,hello,1,0,hello world,-2.460898,-11.80618,-14.267078


All noisy channel models rest upon assumptions about the noise process, i.e. the way that a signal may get distorted. For the present simple spelling corrector, when an error is considered -- say, the non-word *acress* -- it is assumed that it derives from an intended real word by means of a single letter-change (i.e. insertion, *acres* > *acress*; deletion, *actress* > *acress*; substitution, *access* > *acress*; or transposition, *caress* > *acress*). We treat each change as targeting a part of the word that consists of two letters (i.e., a bigraph): for insertion, an extra letter is inserted in the middle of the bigraph part (*es* > *e**s**s*); for deletion, the second letter of the bigraph part is removed (*c**t*** > *c*); for substitution, the first letter of the bigraph part is swapped with something else (*ce* > ***r**e*); and for transposition, the order of the two letters in the bigraph part is swapped (*ca* > ***ac***). 

The error model assumes that a change to an intended word is sampled from the set of valid changes that can be applied across all bigraph parts in that word, through two steps: first, a part is picked; and second, a change is applied to that part. Together, these two steps yield a probability distribution over changes that could have been applied to the intended word (i.e. a joint probability distribution over the bigraph part and result of a change).  
> For the purposes of this assignment, we assume that a part is sampled in proportion to the total number of times we have seen any change applied to that part (in any word), based on a corpus of spelling errors. This is a little different to what we saw in the introductory lecture, where parts were sampled uniformly (i.e., each with equal probability).

It is very important to understand that **the changes are based on the intended real word, not the observed non-word**; for example, the error model probability for the candidate *actress* as a correction of *acress* is based on considering the change that transforms *actress* into *acress* against the backdrop of all other changes that could have applied to *actress*, rather than on considering the change that transforms *acress* into *actress* against the backdrop of all other changes that could have transformed *acress* into a real word. In this way, the error model represents a form of reverse-engineering how an observed non-word could have derived from a real word, if that real word were the intended word.

## Task 1.1: Candidate generation [10 points]

When faced with a non-word, the first task is to find candidate real-word corrections for it.

The codebase for this assignment provides you with a `CandidateGenerator` that can perform this task. The generator finds all candidate real-words that can be obtained within a provided number of letter-changes from a target non-word.

In the setup for this Notebook, you created a simple candidate generator via
```python
simple_candidate_gen = CandidateGenerator(1, "inputs/words.txt")
```

The first argument (`1`) is the maximum number of letter-changes to apply. The second argument (`inputs/words.txt`) is the path to a text file containing real words, which is used to filter the generated candidates.

You can use a generator to generate candidate corrections for a target with the `CandidateGenerator.candidates()` method. This method takes the target as an argument, as in `simple_candidate_gen.candidates("yse")`, and produces a DataFrame containing the candidate real word corrections for that target.

Run the code cell below to generate candidates for the non-word `ot`.

In [4]:
simple_candidate_gen.candidates("ot")

,candidate,distance,paths_to_target,paths_from_candidate
0,t,1,1,78
1,o,1,1,78
2,to,1,1,131
3,at,1,1,131
4,et,1,1,131
5,it,1,1,131
6,ut,1,1,131
7,of,1,1,131
8,oh,1,1,131
9,oi,1,1,131


The output DataFrame contains the candidates in the first column, the distance in number of letter-changes between the target and candidate in the second column, and other information in other columns. Soon, you'll explore what this other information means. But first, think about the number of candidates that are generated, i.e. the number of rows in the DataFrame.

1. Generate candidates for the non-word `hippopotamsu`. How does the number of candidates compare to the number for `ot`? Why do you think this is? **[1 point]**

In [5]:
simple_candidate_gen.candidates("hippopotamsu")

,candidate,distance,paths_to_target,paths_from_candidate
0,hippopotamus,1,1,660


The number of candidates generated for the non-word "hippopotamsu" is much lower than for "ot." Specifically, "ot" generated 33 candidates, while "hippopotamsu" generated only 1 candidate. This difference occurs because "hippopotamsu" is much longer and more specific, with fewer plausible one-letter changes that transform it into a valid real word.

**paths_from_candidate**

Since the noisy channel model works by reverse-engineering changes that might have transformed a candidate word into the target non-word, a candidate's evaluation by an error model will be affected by the *other* changes that might have transformed it into something *other* than the target word; in other words, by other paths that the changes could have taken the candidate along. The `paths_from_candidate` column reports on the total number of paths that changes could have taken a candidate along. For example, there are 131 total paths away from the word `to`; one of these paths leads to the target `ot` (by transposing the letters), but there are also many others, such as a path to `do` (by substituting the t with a d) and a path to `top` (by inserting a p after the o).

The `paths_from_candidate` numbers come from adding together the number of possible changes that could be applied to each bigraph part in the word. The number of changes a bigraph part permits is the sum of the following:
* 26 possible insertion changes, one for each letter that could be inserted  
* 1 possible deletion change that targets the second letter, **except** when that letter is the end-of-word marker `#`, which cannot be deleted  
* 25 possible substitution changes, one for each *other* identity the first letter could take, **except** when that letter is the start-of-word marker `>`, which cannot be substituted for anything else
* 1 possible transposition change that swaps the order of the two letters, **except** when one of them is the start-of-word marker `>` or end-of-word marker `#`, which cannot be reordered, **or** when the two letters are identical, in which case reordering them does not actually change the word  

For example, the word *pool* contains the following bigraph parts, each of which permits the corresponding number of valid changes:  
* `>p`: 26 insertion + 1 deletion + 0 substitution + 0 transposition = 27 valid changes
* `po`: 26 insertion + 1 deletion + 25 substitution + 1 transposition = 53 valid changes
* `oo`: 26 insertion + 1 deletion + 25 substitution + 0 transposition = 52 valid changes
* `ol`: 26 insertion + 1 deletion + 25 substitution + 1 transposition = 53 valid changes
* `l#`: 26 insertion + 0 deletion + 25 substitution + 0 transposition = 51 valid changes

This yields a total of 27 + 53 + 52 + 53 + 51 = 236 valid paths away from the real word *pool*. Each path is the result of applying a single change to a single bigraph part; therefore, each path leads to a sequence of characters that is one letter-change away from *pool*. Some paths will lead to real words and some paths will lead to non-words; some paths might even lead to the same result, obtained through different means (e.g., there are two paths that lead from *pool* to *pol*, one that deletes the first *o* and one that deletes the second *o*).

Candidates can have different numbers of paths, depending on the kinds of changes that are applicable to the candidate.

2. Identify the pattern in the `paths_from_candidate` column for the DataFrame for `ot` generated above. What do you think is the major factor behind that pattern? **[1 point]**

The pattern in the paths_from_candidate column for the DataFrame generated for "ot" shows that shorter candidates (like single letters "t" or "o") generally have fewer paths, while longer candidates (such as "cot," "dot," "got") have more paths. Longer words have more bigraph parts, and each bigraph permits a certain number of valid changes. Consequently, as the word length increases, there are more possible insertion, deletion, substitution, and transposition changes across the different bigraphs

**paths_to_target**

When we evaluate a candidate with an error model, we need to consider all of the ways that changes could transform it into the target. The `paths_to_target` column counts how many ways there are to transform the candidate into the target, within the designated number of changes.

3. Generate candidates for the non-word `acress`. What is different about the `paths_to_target` value for the candidate `acres`? Why do you think that is? **[1 point]**

In [6]:
simple_candidate_gen.candidates("acress")

,candidate,distance,paths_to_target,paths_from_candidate
0,cress,1,1,289
1,acres,1,2,290
2,caress,1,1,342
3,access,1,1,341
4,across,1,1,342
5,actress,1,1,395


The paths_to_target value for the candidate "acres" is different because it is 2, whereas all other candidates have a value of 1. This indicates that there are two different ways to transform "acres" into the non-word "acress," which might be associated with the position of inserting 's'.

**Influence on spelling correction**

The candidate generator has an influence on the whole spelling correction system. This can easily be seen in the simple spelling corrector we have set up, in which the error model treats all changes as equally likely, and the language model treats all words as equally likely. 

The idea that the error model treats all changes as equally likely can be thought of as equivalent to having observed each possible change from each possible bigraph once in a corpus of errors. Since our spelling corrector assumes that the probability of picking a bigraph part in a word as the target of a change is proportional to the number of times we have seen changes to that bigraph (in any word) in the corpus, using an error model in which all changes are equally likely means that the probability of picking a part is proportional to the number of valid changes that part permits. For example, in the case of *pool* described above, the probability of picking the part `>p` is given by 27/236, since there are 27 valid changes that can be applied to `>p` out of 236 valid changes total that can be applied to the word. Since each of the 27 changes that can be applied to this part are given equal probability, each pathway that results from applying one of them gets 1/27 of this probability mass, for a total probability of 1/236. 

As a consequence of using this error model and language model, any differences you observe between scores of candidates are based entirely on the `paths_from_candidate` and `paths_to_target` values.

The spelling corrector can be used to correct a single target non-word via
```python
simple_corrector.correct_word(target_nonword, 0)
```
where `target_nonword` should be replaced with a string representing the target non-word.

4. Generate corrections for the non-word `ot`. Why are the best candidates best? What does this tell you about how candidate generation can affect spelling correction? **[2 points]**

In [7]:
simple_corrector.correct_word("ot", 0)

,target,candidate,distance,error_score,lm_score,score
0,ot,t,1,-1.892095,-1.20412,-3.096215
1,ot,o,1,-1.892095,-1.20412,-3.096215
2,ot,oi,1,-2.117271,-1.20412,-3.321391
3,ot,oz,1,-2.117271,-1.20412,-3.321391
4,ot,ox,1,-2.117271,-1.20412,-3.321391
5,ot,ow,1,-2.117271,-1.20412,-3.321391
6,ot,op,1,-2.117271,-1.20412,-3.321391
7,ot,on,1,-2.117271,-1.20412,-3.321391
8,ot,or,1,-2.117271,-1.20412,-3.321391
9,ot,oh,1,-2.117271,-1.20412,-3.321391


The best candidates for the non-word "ot" are "t" and "o," both of which have the highest scores (less negative) of -3.096215. These candidates have a better score because they have higher paths_to_target values and relatively fewer paths_from_candidate values compared to other candidates. This indicates that candidates with fewer paths away from them are more likely to be selected. 

5. Generate corrections for the non-word `acress`. Compare the scores for `acres` and `cress`. Which one is better, and why? What does this tell you about how candidate generation can affect spelling correction? **[2 points]**

In [8]:
simple_corrector.correct_word("acress", 0)

,target,candidate,distance,error_score,lm_score,score
0,acress,acres,1,-2.161368,-1.20412,-3.365488
1,acress,cress,1,-2.460898,-1.20412,-3.665018
2,acress,access,1,-2.532754,-1.20412,-3.736874
3,acress,caress,1,-2.534026,-1.20412,-3.738146
4,acress,across,1,-2.534026,-1.20412,-3.738146
5,acress,actress,1,-2.596597,-1.20412,-3.800717


The candidate "acres" has a better score (-3.365488) compared to "cress" (-3.665018). The reason "acres" is ranked higher is due to its higher paths_to_target value and a relatively lower paths_from_candidate value compared to "cress." This comparison highlights that the system prioritizes candidates that can be reached through more error pathways (higher paths_to_target), which increases their likelihood under the error model. 

**Candidate generation beyond 1 change**

The code cell below creates a spelling corrector in which the candidate generator is allowed to generate candidates that are 1 or 2 letter-changes away from the target non-word. This means that more candidates are generated.

In [9]:
twochange_candidate_gen = CandidateGenerator(2, "inputs/words.txt")
twochange_corrector = SpellCorrector(twochange_candidate_gen, uniform_error_model, uniform_language_model, noerror_rate=0)

6. Use this new spelling corrector to generate corrections for `acress`, as before. What do you notice about the error model scores for the new candidates? Why do you think this is, and what does it tell you about how candidate generation can affect spelling correction? **[2 points]**

In [10]:
twochange_corrector.correct_word("acress", 0)

,target,candidate,distance,error_score,lm_score,score
0,acress,acres,1,-2.240549,-1.20412,-3.444669
1,acress,cress,1,-2.540079,-1.20412,-3.744199
2,acress,access,1,-2.611936,-1.20412,-3.816056
3,acress,caress,1,-2.613207,-1.20412,-3.817327
4,acress,across,1,-2.613207,-1.20412,-3.817327
5,acress,actress,1,-2.675778,-1.20412,-3.879898
6,acress,acers,2,-4.924796,-1.20412,-6.128916
7,acress,aces,2,-4.925588,-1.20412,-6.129708
8,acress,ares,2,-4.925588,-1.20412,-6.129708
9,acress,press,2,-5.097887,-1.20412,-6.302007


The error model scores for candidates such as "acres" and "cress" are slightly less negative compared to the earlier results produced by simple_corrector. This shift in error model scores occurs because the twochange_corrector allows for generating corrections that require two edits, thus expanding the set of potential candidates.

7. Given the observation you just made, under what circumstances do you think a candidate that is more than 1 letter-change away from a target might be chosen as the best correction for a non-word, over candidates that are only 1 letter-change away? **[1 point]**
   
   *Hint: remember that the error model is not the only component of the spelling corrector!*

This can happen if the two-change candidate forms a more common or contextually appropriate word compared to the one-change candidates. The higher likelihood of the word in natural language use can outweigh the penalty from the greater edit distance.

## Task 1.2: the error model [4 points]

Each candidate needs to be evaluated according to how likely it is to have been transformed into the target via spelling errors. For this, we use an error model.

The codebase for this assignment provides you with an `ErrorModel` that can be used to create an error model. The model is based on counts of how often various possible insertion, deletion, substitution, and transposition changes occurred in a particular dataset. The simple spelling corrector we used before contained an error model trained on a *uniform* dataset that assumes all changes could occur equally often. Here, we will explore an error model trained on a dataset of actual change occurrences, provided by Peter Norvig.

The code cell below creates the new error model, and a new spelling corrector that incorporates it.

In [11]:
error_model = ErrorModel("inputs/errors_corpus.txt")
candidate_gen = CandidateGenerator(1, "inputs/words.txt", error_model)
em_corrector = SpellCorrector(candidate_gen, error_model, uniform_language_model, noerror_rate=0)

1. Using the new spelling corrector, generate corrections for `acress`. How do the rankings of candidates other than `acres` compare to previously, and why do you think that is? What does that tell you about the interaction of the error model and the candidate generator? **[2 points]**

In [12]:
error_model = ErrorModel("inputs/errors_corpus.txt")
candidate_gen = CandidateGenerator(1, "inputs/words.txt", error_model)
em_corrector = SpellCorrector(candidate_gen, error_model, uniform_language_model, noerror_rate=0)

corrections = em_corrector.correct_word("acress", 0)

print(corrections)

   target candidate  distance  error_score  lm_score     score
0  acress     acres         1    -1.408591  -1.20412 -2.612711
1  acress   actress         1    -2.085023  -1.20412 -3.289143
2  acress    across         1    -2.106361  -1.20412 -3.310481
3  acress    caress         1    -2.425697  -1.20412 -3.629817


Compared to previous outputs, "actress" and "across" now rank higher than before. This is because the new error model, trained on real data, assigns more realistic probabilities to certain types of spelling changes based on observed error patterns. 
This shows a well-trained error model can prioritize candidates that reflect more likely human spelling mistakes, thereby improving the accuracy of the spelling corrector.

As shown in the code cell above, the error model can also be provided as an argument when constructing the candidate generator, so that candidates requiring changes that were not observed in the training data are not generated. This may have implications for the paths associated with candidate generation.

2. Using the new candidate generator `candidate_gen`, generate candidates for the non-word `acress` that filter out paths containing unobserved changes. Compare the numbers in both path columns to what you saw under the original candidate generator. How have they changed? Drawing on the observations you made earlier about the influences of paths on error probabilities, what does this tell you about the interaction of the error model and the candidate generator? **[2 points]**

In [13]:
# Generate candidates for 'acress' with the new candidate generator that filters out unobserved changes
filtered_candidates = candidate_gen.candidates("acress")
print(filtered_candidates)

  candidate  distance  paths_to_target  paths_from_candidate
0     acres         1                2                    80
1    caress         1                1                    88
2    across         1                1                    74
3   actress         1                1                   103


There are updated path counts for candidates of "acress." Specifically, "acres" has 2 valid paths to it, while "caress," "across," and "actress" each have only 1. By filtering out unlikely paths, the candidate generator enhances the accuracy of the spelling corrector.

## Task 1.3: the language model [6 points]

Each candidate also needs to be evaluated for how good a sentence it creates when substituted for the target. For this, we use a language model.

The codebase for this assignment includes the `run_srilm()` function you saw in Assignment 1. You can use this function to train and save language models, e.g.
```python
run_srilm("ngram-count -order 1 -unk -text inputs/train_100k.txt.gz -lm models/100k_unigram.lm")
```

The language model can be read into python using the `arpa.loadf()` method, as follows:
```python
unigram_language_model = arpa.loadf("models/100k_unigram.lm")[0]
```
The argument is the path to the language model file. The final `[0]` is required because the method outputs a list of models, which (for models trained with SRILM) will only contain one item.

Once a language model has been read, it can be incorporated into the spelling corrector by replacing the appropriate argument, e.g.
```python
unigram_corrector = SpellCorrector(candidate_gen, error_model, unigram_language_model, noerror_rate=0)
```

The new corrector can be used to generate corrections for a word in sentential context, e.g.
```python
unigram_corrector.correct_word("she is a comic acress with many awards to her name", 4)
```

1. Train a unigram language model on the file `train_100k.txt.gz` and use it to create a new spelling corrector. Using that corrector, generate corrections for the word `acress` in the sentence `she is a comic acress with many awards to her name`. How do these corrections compare to those previously generated, with just the error model? Can you explain the similarities and/or differences in rankings of candidates across the two correctors? **[2 points]**

In [14]:
# Train the unigram language model
run_srilm("ngram-count -order 1 -unk -text inputs/train_100k.txt.gz -lm models/100k_unigram.lm")

# Load the trained unigram language model
unigram_language_model = arpa.loadf("models/100k_unigram.lm")[0]

# Create a new spelling corrector using the unigram language model
unigram_corrector = SpellCorrector(candidate_gen, error_model, unigram_language_model, noerror_rate=0)

# Generate corrections for the word 'acress' in context
unigram_corrections = unigram_corrector.correct_word("she is a comic acress with many awards to her name", 4)

print(unigram_corrections)

The command ran successfully
   target candidate  distance  position  \
0  acress    across         1         4   
1  acress   actress         1         4   
2  acress     acres         1         4   
3  acress    caress         1         4   

                                            sentence  error_score   lm_score  \
0  she is a comic across with many awards to her ...    -2.106361 -66.740294   
1  she is a comic actress with many awards to her...    -2.085023 -68.730368   
2  she is a comic acres with many awards to her name    -1.408591 -69.415214   
3  she is a comic caress with many awards to her ...    -2.425697 -70.328666   

       score  
0 -68.846655  
1 -70.815391  
2 -70.823805  
3 -72.754363  


The scores have become more negative overall. While the error model focuses on the likelihood of transformations from the misspelled word to the candidates based on observed spelling errors, the unigram model assesses how likely the candidates are in the context of the surrounding sentence.

2. Train a bigram language model on the file `train_100k.txt.gz` and use it to create another new spelling corrector. Using that corrector, repeat the above investigation. What has changed, and why? **[2 points]**

In [16]:
# Train the bigram language model
run_srilm("ngram-count -order 2 -unk -text inputs/train_100k.txt.gz -lm models/100k_bigram.lm")

# Load the trained bigram language model
bigram_language_model = arpa.loadf("models/100k_bigram.lm")[0]

# Create a new spelling corrector using the bigram language model
bigram_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0)

# Generate corrections for the word 'acress' in context
bigram_corrections = bigram_corrector.correct_word("she is a comic acress with many awards to her name", 4)

print(bigram_corrections)

The command ran successfully
   target candidate  distance  position  \
0  acress   actress         1         4   
1  acress    across         1         4   
2  acress     acres         1         4   
3  acress    caress         1         4   

                                            sentence  error_score   lm_score  \
0  she is a comic actress with many awards to her...    -2.085023 -55.475186   
1  she is a comic across with many awards to her ...    -2.106361 -59.051406   
2  she is a comic acres with many awards to her name    -1.408591 -60.309881   
3  she is a comic caress with many awards to her ...    -2.425697 -61.620077   

       score  
0 -57.560209  
1 -61.157767  
2 -61.718472  
3 -64.045774  


The lm_scores are less negative compared to the unigram model, indicating better contextual fit. The key difference lies in the bigram model's ability to evaluate the likelihood of word pairs. 

3. Compare the two corrections from the two language models for the non-word `acress` in the sentence `she is a talented acress who is known for her comedic timing`. How is the pattern similar to or different from the comparison you just did? Why do you think that is? **[2 points]**

   *Hint: think about the fact that the language model score is ultimately based on a corpus of limited size, and remember that language models trained with SRILM use backoff by default.*

In [17]:
# Generate corrections for 'acress' in a new sentence
new_unigram_corrections = unigram_corrector.correct_word("she is a talented acress who is known for her comedic timing", 4)
new_bigram_corrections = bigram_corrector.correct_word("she is a talented acress who is known for her comedic timing", 4)

# Display the results for the new sentence
print("Unigram Corrections:", new_unigram_corrections)
print("Bigram Corrections:", new_bigram_corrections)

Unigram Corrections:    target candidate  distance  position  \
0  acress    across         1         4   
1  acress   actress         1         4   
2  acress     acres         1         4   
3  acress    caress         1         4   

                                            sentence  error_score   lm_score  \
0  she is a talented across who is known for her ...    -2.106361 -79.794828   
1  she is a talented actress who is known for her...    -2.085023 -81.784902   
2  she is a talented acres who is known for her c...    -1.408591 -82.469748   
3  she is a talented caress who is known for her ...    -2.425697 -83.383200   

       score  
0 -81.901189  
1 -83.869925  
2 -83.878339  
3 -85.808897  
Bigram Corrections:    target candidate  distance  position  \
0  acress    across         1         4   
1  acress   actress         1         4   
2  acress     acres         1         4   
3  acress    caress         1         4   

                                            sentenc

Both the unigram and bigram models rank the candidates for "acress" similarly, with "actress" leading. However, the bigram model's scores are less negative, indicating a better contextual fit. This difference arises because the bigram model accounts for word pairs, enhancing its understanding of context compared to the unigram model.

## Task 1.4: combining models [5 points]

A spelling corrector balances the error model and the language model. Often, one model will prefer one candidate, and the other model will prefer another candidate. The choice between the candidates then comes down to how the models are combined.

When combining models, we can choose to weight the language model up or down relative to the error model, to offset the balance between them. In the `SpellCorrector` object, this is achieved through the keyword argument `lm_doublings`, which represents the number of times the language model should be doubled in importance relative to the error model. When `lm doublings` is positive, the language model takes on additional weight relative to the error model; when `lm_doublings` is negative, the language model loses weight relative to the error model, and when `lm_doublings` is zero, both models are weighted equally. 

1. The spelling correctors we have been using so far have a slight bias toward the language model. The code cell below defines one with an extreme bias toward the error model. Compare corrections of your bigram-based corrector and this new corrector for the non-word `mne` in the sentence `if i had a gold mne my life would be easy`. How are they different, and why? **[2 points]**

In [18]:
extreme_bias_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0, lm_doublings=-6)
bigram_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0)

extreme_bias_correction = extreme_bias_corrector.correct_word("if i had a gold mne my life would be easy", 5)
bigram_correction = bigram_corrector.correct_word("if i had a gold mne my life would be easy", 5)

print("Extreme bias corrector output:", extreme_bias_correction)
print("Bigram-based corrector output:", bigram_correction)

Extreme bias corrector output:   target candidate  distance  position  \
0    mne       men         1         5   
1    mne      mine         1         5   
2    mne      mane         1         5   
3    mne        me         1         5   

                                     sentence  error_score  lm_score     score  
0   if i had a gold men my life would be easy    -1.744293 -0.433346 -2.177639  
1  if i had a gold mine my life would be easy    -1.793169 -0.394553 -2.187722  
2  if i had a gold mane my life would be easy    -1.775511 -0.462009 -2.237520  
3    if i had a gold me my life would be easy    -2.095169 -0.403152 -2.498322  
Bigram-based corrector output:   target candidate  distance  position  \
0    mne      mine         1         5   
1    mne        me         1         5   
2    mne       men         1         5   
3    mne      mane         1         5   

                                     sentence  error_score   lm_score  \
0  if i had a gold mine my life would 

The extreme bias corrector prioritizes the error model, suggesting candidates like "gould" and "golds" for "mne," which are phonetically closer to the misspelled word. In contrast, the bigram corrector would likely provide contextually relevant suggestions based on language model scores. This difference highlights the trade-off between spelling similarity and grammatical coherence in the corrections.

2. Now make a new corrector that is exactly the same as the previous one, except with a less extreme error model bias, by setting `lm_doublings` to -3. Compare the rankings of `men` and `me` in the new model to the previous models. How are they different, and why? What does this tell you about how the error model and language model work together? **[2 points]**

In [19]:
extreme_bias_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0, lm_doublings=-6)
bigram_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0)
less_extreme_bias_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0, lm_doublings=-3)

extreme_bias_correction = extreme_bias_corrector.correct_word("if i had a gold mne my life would be easy", 5)
bigram_correction = bigram_corrector.correct_word("if i had a gold mne my life would be easy", 5)
less_extreme_bias_correction = less_extreme_bias_corrector.correct_word("if i had a gold mne my life would be easy", 5)

print("Extreme bias corrector output:", extreme_bias_correction)
print("Bigram-based corrector output:", bigram_correction)
print("Less extreme bias corrector output:", less_extreme_bias_correction)

Extreme bias corrector output:   target candidate  distance  position  \
0    mne       men         1         5   
1    mne      mine         1         5   
2    mne      mane         1         5   
3    mne        me         1         5   

                                     sentence  error_score  lm_score     score  
0   if i had a gold men my life would be easy    -1.744293 -0.433346 -2.177639  
1  if i had a gold mine my life would be easy    -1.793169 -0.394553 -2.187722  
2  if i had a gold mane my life would be easy    -1.775511 -0.462009 -2.237520  
3    if i had a gold me my life would be easy    -2.095169 -0.403152 -2.498322  
Bigram-based corrector output:   target candidate  distance  position  \
0    mne      mine         1         5   
1    mne        me         1         5   
2    mne       men         1         5   
3    mne      mane         1         5   

                                     sentence  error_score   lm_score  \
0  if i had a gold mine my life would 

The less extreme bias corrector ranks contextually appropriate candidates like "men" and "me" higher compared to the extreme bias corrector. The extreme bias model favors phonetic similarities (e.g., "gould," "golds"), while the less extreme model incorporates context more effectively.

When considering entire sentences, it's not always possible to know where a spelling error is, or even if there is one, because many spelling errors create real words. The spelling corrector handles this by considering whether each word is or is not an error, based on a parameter that defines how likely it is for a word *not* to be an error. In this codebase, that is handled through the keyword argument `noerror_rate`, passed when the spelling corrector is created.

The code cell below shows the use of the `noerror_rate` argument, as well as a method `SpellCorrector.correct_sentence()` that tabulates possible corrections for up to one word anywhere in a sentence. *Note: you will need to replace `bigram_language_model` with the name of your bigram language model for the code to run.*

In [20]:
sentence_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=0.995, lm_doublings=0)
sentence_corrector.correct_sentence("a rolling stone gathers no moss")

,target,candidate,position,sentence,error_score,lm_score,score
0,--,--,--,a rolling stone gathers no moss,-0.002177,-22.188404,-22.190581
1,moss,mass,5,a rolling stone gathers no mass,-3.952093,-20.385922,-24.338015
2,moss,moses,5,a rolling stone gathers no moses,-3.284943,-21.152062,-24.437005
3,moss,most,5,a rolling stone gathers no most,-3.951476,-20.947566,-24.899042
4,no,not,4,a rolling stone gathers not moss,-3.243991,-21.821536,-25.065527
5,no,now,4,a rolling stone gathers now moss,-2.786346,-22.446007,-25.232353
6,gathers,gather,3,a rolling stone gather no moss,-4.229170,-21.408652,-25.637822
7,stone,stones,2,a rolling stones gathers no moss,-3.029113,-22.898802,-25.927915
8,a,as,0,as rolling stone gathers no moss,-2.868016,-23.374372,-26.242388
9,no,nor,4,a rolling stone gathers nor moss,-3.084714,-23.206808,-26.291522


3. In the case of the sentence `a rolling stone gathers no moss`, a `noerror_rate` of 0.995 leads to the correct observation that there are no spelling errors. For the sentence `the big hose sits on a tiny lot`, this same value also leads to the prediction that there are no spelling errors, even though `hose` should be corrected to `house`! Experiment with different values of `noerror_rate` to find a few values of `noerror_rate` that yield the correct observation for both sentences (do not change any other aspects of the spelling corrector). What happens if you make the `noerror_rate` *too* low? **[1 point]**

In [21]:
noerror_rates = [0.9, 0.5, 0.1]  

for rate in noerror_rates:
    sentence_corrector = SpellCorrector(candidate_gen, error_model, bigram_language_model, noerror_rate=rate, lm_doublings=0)
    print(f"\nTesting noerror_rate = {rate}")
    
    # Test both sentences
    result1 = sentence_corrector.correct_sentence("a rolling stone gathers no moss")
    result2 = sentence_corrector.correct_sentence("the big hose sits on a tiny lot")
    
    print("\nCorrections for 'a rolling stone gathers no moss':")
    print(result1)
    
    print("\nCorrections for 'the big hose sits on a tiny lot':")
    print(result2)


Testing noerror_rate = 0.9

Corrections for 'a rolling stone gathers no moss':
     target candidate position                          sentence  error_score  \
0        --        --       --   a rolling stone gathers no moss    -0.045757   
1      moss      mass        5   a rolling stone gathers no mass    -2.651063   
2      moss     moses        5  a rolling stone gathers no moses    -1.983913   
3      moss      most        5   a rolling stone gathers no most    -2.650446   
4        no       not        4  a rolling stone gathers not moss    -1.942961   
5        no       now        4  a rolling stone gathers now moss    -1.485316   
6   gathers    gather        3    a rolling stone gather no moss    -2.928140   
7     stone    stones        2  a rolling stones gathers no moss    -1.728083   
8         a        as        0  as rolling stone gathers no moss    -1.566986   
9        no       nor        4  a rolling stone gathers nor moss    -1.783684   
10        a        at        

Experimenting with the noerror_rate shows that values around 0.995 correctly identify no errors in "a rolling stone gathers no moss," while lower values like 0.8 correctly identify "hose" as an error in "the big hose sits on a tiny lot"; however, setting it too low can lead to excessive false positives.

# Part 2: Minimum edit distance [25 points]

Many noisy channel models require finding the best path through a series of steps, such as the best sequence of words for a sentence where all words may or may not contain spelling errors. The general algorithm for finding such a path is known as the Viterbi algorithm. A related algorithm, the minimum edit distance algorithm, can be used to find the smallest number of letter-changes that are required to turn one string into another, and identify what these letter-changes are, both of which are important for a noisy channel spelling corrector.

In this part of the assignment, you will implement an algorithm to find the minimum edit distance between two strings and extend it to consider new error costs. You will not complete the step of tracing back to identify the required sequence of letter-changes (not because it is too difficult, but because it requires rewriting all of your functions to keep track of additional structure, which would create a lot of extra work).

## Setup

Run the code cell below to import the packages and autotesting functions for this part.

In [22]:
import math
from tester import test

## Task 2.1: edit cost [4 points]

As discussed in class, the minimum edit distance algorithm operates in a stepwise manner, with each step identifying the smallest number of letter-changes from a *source* substring to a *target* substring. In this framework, a letter-change is known as an *edit*, and each kind of edit is associated with a particular *cost*.

Each edit represents a movement from a *previous* (source, target) substring pair to a *new* (source, target) substring pair. In this movement, one or both of the source and target will gain a new character appended at their right edge. We will at first consider three kinds of edits, represented by three different kinds of movements:  
- **insertion**: source stays the same, target gains a character; e.g. (*inte*, *exec*) > (*inte*, *execu*). This is insertion because the best path from the previous source to previous target can still be used, as long as the gained character (*u*) is then inserted at the end of the previous target.  
- **deletion**: source gains a character, target stays the same; e.g. (*inte*, *exec*) > (*inten*, *exec*). This is deletion because the best path from the previous source to previous target can still be used, as long as the gained character (*n*) is removed from the end of the new source first.  
- **substitution**: source and target both gain a character; e.g. (*inte*, *exec*) > (*inten*, *execu*). This is really a kind of double-insertion or double-deletion: the best path from the previous source to previous target can still be used, as long as the gained characters (*n*, *u*) are also added to or deleted from the end of the new source and target. We call it substitution because the same thing happens to both the source and the target, except with one gained character substituted for another.

We will consider standard costs for these edits, which gives rise to the *Levenshtein distance* between two strings.  
- **insertion**: costs 1 (no matter what)  
- **deletion**: costs 1 (no matter what)  
- **substitution**: costs 0 if the gained characters are identical; otherwise 2  

Complete the function `lev_edit_cost()` that takes as arguments a previous (source, target) substring pair and a new (source, target) substring pair, identifies the edit associated with the movement from the previous pair to the new pair, and returns the cost of that edit.

In [23]:
def lev_edit_cost(prev_pair, new_pair):
    """Returns the cost associated with an edit, identified by comparing a previous
    (source, target) substring pair with a new (source, target) substring pair.
    Assumes Levenshtein costs: 1 for insertion and deletion, and 0/2 for substitution.
    """
    prev_source, prev_target = prev_pair
    new_source, new_target = new_pair
    
    # Check if an insertion occurred
    if len(new_source) == len(prev_source) and len(new_target) == len(prev_target) + 1:
        return 1  # Insertion cost
    # Check if a deletion occurred
    elif len(new_source) == len(prev_source) + 1 and len(new_target) == len(prev_target):
        return 1  # Deletion cost
    # Check for substitution
    elif len(new_source) == len(prev_source) + 1 and len(new_target) == len(prev_target) + 1:
        if new_source[-1] == new_target[-1]:
            return 0  # No cost for same character
        else:
            return 2  # Cost for different characters
    return math.inf  # Not a valid edit

To check whether your function works as expected, run the following code cell.  

In [24]:
test(lev_edit_cost)

Task 2.1: CORRECT. Points: 4/4


## Task 2.2: step distance [4 points]

At each step in the algorithm, to find the shortest distance between a new (source, target) pair, the edits from three different previous (source, target) pairs are considered. For each edit, the shortest distance between the corresponding previous (source, target) is already known. The distance to the new (source, target) pair that makes use of that edit is obtained by adding the cost of the edit to the known distance between the (source, target) pair. Each edit gives rise to a distance, and the smallest of these distances represents the shortest distance between the new (source, target) pair.

Complete the function `step_distance()`, which compares the distances using the three paths to a new (source, target) pair and returns the lowest one. The function takes as arguments a new (source, target) string pair (e.g. `("inte", "exec")`), as well as a list of 3 tuples (in general, there may be more than 3, so make sure you iterate over this list rather than unpacking it). Each tuple has two items: firstly, a possible previous (source, target) string pair; and secondly, the known shortest distance between that previous source and target. For example, the list may look like `[(("int", "exec"), 7), (("inte", "exe"), 5), (("int", "exe"), 6)]`.

The `step_distance()` function has a keyword argument `edit_cost` that represents an edit cost function. When calculating edit costs, you should use `edit_cost(...)` rather than directly using a particular edit cost function such as `lev_edit_cost(...)`, so that `step_distance()` can be used with different edit costs easily.

In [25]:
def step_distance(new_pair, possible_prev_steps, edit_cost=lev_edit_cost):
    """Calculates the minimum distance to a new (source, target) pair
    based on the previous pairs and their known distances.
    """
    min_distance = math.inf
    for prev_pair, distance in possible_prev_steps:
        cost = edit_cost(prev_pair, new_pair)
        min_distance = min(min_distance, distance + cost)
    return min_distance

To check whether your function works as expected, run the following code cell.  

In [26]:
test(step_distance)

Task 2.2: CORRECT. Points: 4/4


## Task 2.3: initializing the search table [2 points]

As the minimum edit algorithm proceeds, it fills in cells in a large table. For efficiency reasons, it is useful to construct this table in advance. Some cells are filled in automatically as part of the initialization process; others can be given dummy values to be overwritten when the algorithm runs.

The size of the table is determined by the size of the source and target words. You may assume that each word has already had a start symbol added at the beginning. The table should have one row for every character in the source word (including the start symbol), and one column for every character in the target word (including the start symbol).

The first row and column of the table are filled in as part of the initialization process. The first row should consist of the numbers 0 through to the length of the target, minus one (i.e. excluding the start symbol). The first column should consist of the numbers 0 through to the length of the source, minus one (i.e. excluding the start symbol).

Complete the function `init_search_table()` that takes source and target strings as arguments and returns a list of lists representing the initialized table. Each list inside the list-of-lists represents a row in the table; values at the same position in different lists represent a column in the table. For example, `init_search_table(">no", ">yes")` should return `[[0, 1, 2, 3], [1, None, None, None], [2, None, None, None]]`

In [27]:
def init_search_table(source, target):
    
    table = [[None] * len(target) for _ in range(len(source))]

    for j in range(len(target)):
        table[0][j] = j 

    for i in range(len(source)):
        table[i][0] = i  
        
    return table

Run the code cell below to test your function on the example `init_search_table(">no", ">yes")`. The tester function converts your table into a pandas DataFrame so that it can be read more easily (note that `NaN` is the pandas equivalent of `None`).

In [28]:
test(init_search_table)

Test input: init_search_table(">no", ">yes")
Your output:

   >    y    e    s
>  0  1.0  2.0  3.0
n  1  NaN  NaN  NaN
o  2  NaN  NaN  NaN

Task 2.3: CORRECT. Points: 2/2


## Task 2.4: filling in the search table [5 points]

The minimum edit distance algorithm fills in a search table one cell at a time, based on consideration of the ways of stepping to that cell from the left, above, or diagonally-left above. This consideration draws upon the values already in the table, as well as the corresponding substrings of the source and target strings (which are assumed to have already had the start symbol added at the beginning).

The cell `table[i][j]` represents the shortest distance between the substrings (`source[:i+1]`, `target[:j+1]`). The value for this cell can be determined from the `step_distance()` function, based on consideration of the substrings and shortest distances corresponding to `table[i-1][j-1]`, `table[i-1][j]`, and `table[i][j-1]`. Since the first row and column were filled in at initialization, the three required shortest distances will be known if the rows and columns are iterated over in consecutive (increasing index) order.

Complete the function `complete_search_table()`, which takes as arguments a source string, a target string, and an initialized search table, and returns a completed search table. The function also takes a keyword argument `edit_cost` representing a function used to calculate the costs of edits, which should be passed along when calling `step_distance()`.

In [29]:
def complete_search_table(source, target, table, edit_cost=lev_edit_cost):
    for i in range(1, len(source)):
        for j in range(1, len(target)):
            possible_prev_steps = [
                ((source[:i], target[:j]), table[i-1][j-1]),  
                ((source[:i], target[:j+1]), table[i-1][j]),  
                ((source[:i+1], target[:j]), table[i][j-1])   
            ]

            table[i][j] = step_distance((source[:i+1], target[:j+1]), possible_prev_steps, edit_cost=edit_cost)

    return table

Run the code cell below to test your function with `">intention"` as the source and `">execution"` as the target. The tester function converts your table into a pandas DataFrame so that it can be read more easily.

In [30]:
test(complete_search_table)

Test input: complete_search_table(">intention", ">execution", table), where table is the correctly initialized table for this source and target
Your output:

   >  e  x   e   c   u   t   i   o   n
>  0  1  2   3   4   5   6   7   8   9
i  1  2  3   4   5   6   7   6   7   8
n  2  3  4   5   6   7   8   7   8   7
t  3  4  5   6   7   8   7   8   9   8
e  4  3  4   5   6   7   8   9  10   9
n  5  4  5   6   7   8   9  10  11  10
t  6  5  6   7   8   9   8   9  10  11
i  7  6  7   8   9  10   9   8   9  10
o  8  7  8   9  10  11  10   9   8   9
n  9  8  9  10  11  12  11  10   9   8

Task 2.4: CORRECT. Points: 5/5


## Task 2.5: putting it all together [2 points]

Once a search table has been filled in, the minimum edit distance between the strings is the value in the bottom-right corner.

Complete the function `min_edit_distance()`, which takes as arguments a source string, a target string, and a keyword argument `edit_cost` representing a function used to calculate edit costs, and returns the minimum edit distance from the source to the target string under the edit cost function. The first thing your function will need to do is add a start symbol before the source and target strings; you should use `>` for this (note this is different to `#` as used in lectures and readings). Once the strings have the start symbol, you should be able to use the functions `init_search_table()` and `complete_search_table()` that you wrote previously.

*Note: when you call the `complete_search_table()` function, you should also pass along the `edit_cost` keyword argument, via `complete_search_table(..., edit_cost=edit_cost)`. This ensures that the intended edit cost function is used, and substituting a different function inside `min_edit_distance()` will also cause a different function to be substituted in `complete_search_table()`.*

In [ ]:
def min_edit_distance(source, target, edit_cost=lev_edit_cost):
    # TODO: complete this function
    pass

If your function works, it should tell you that the minimum edit distance between `"intention"` and `"execution"` is 8. The code cell below will check this for you.

In [ ]:
test(min_edit_distance)

## Task 2.6: a new edit cost function [3 points]

In the noisy channel spelling corrector, we generated candidates that were 1 insertion, deletion, substitution, or transposition away from an observed non-word. Ideally, we would like to be able to look at a pair of words and say whether one could be a candidate correction for the other. To do this, we need to know if the edit distance between the words is 1.

However, the current edit costs will not suffice for this purpose, for two reasons:  
1. we have no way of accounting for transpositions: the edit distance of `caress` and `acress` should be 1, but it is 2, because it is treated as an insertion and a deletion  
2. the cost of substitution is too high: the edit distance of `access` and `acress` should be 1, but it is 2  

To account for (1), we need to think about what transposition means in this context: it means that the last two characters in the source substring and the last two characters in the target substring are the same, but in reversed order. To assess this possibility in the framework of moving from a previous (source, target) pair to a new (source, target) pair, we will have to consider the case where the source and target both gain *two* characters (not just the *one* we were considering before). Given this consideration, we account for transposition as follows:  
- **transposition**: source and target both gain *two* characters; e.g. (*li*, *to*) > (*lisp*, *tops*). The best path from the previous source to previous target can still be used, as long as the gained characters (*sp*, *ps*) are also added to or deleted from the end of the new source and target. Like substitution, it is really a form of multiple-insertion or -deletion, where the characters added and deleted have a particular relation to each other.

To account for this addition and (2), we can adopt a different edit cost function, which gives rise to the *Restricted Damerau-Levenshtein* distance.  
- **insertion**: costs 1 (no matter what)  
- **deletion**: costs 1 (no matter what)  
- **substitution**: costs 0 if source and target gain the same single character; otherwise **1**  
- **transposition**: costs 1 if the source and target gain the same two characters in opposite orders; otherwise `math.inf` (infinite cost is our representation for the step being disallowed) 

Complete the function `rdl_edit_cost()` that takes as arguments a previous (source, target) substring pair and a new (source, target) substring pair, identifies the edit associated with the movement from the previous pair to the new pair, and returns the cost of that edit, under the Restricted Damerau-Levenshtein scheme outlined above. 

In [ ]:
def rdl_edit_cost(prev_pair, new_pair):
    # TODO: complete this function
    pass

To check whether your function works as expected, run the following code cell.  

In [ ]:
test(rdl_edit_cost)

## Task 2.7: filling in the search table with transpositions [3 points]

Incorporating transpositions means that, when filling in the search table, we have to consider the possibility that the shortest distance between the substrings (`source[:i+1]`, `target[:j+1]`), as represented in the cell `table[i][j]`, comes from the substrings and shortes distance corresponding to the cell `table[i-2][j-2]`. This means that a *fourth* possible previous (source, target) step has to be included in the list of options provided to `step_distance()` for consideration.

Create a new search table completion function, `complete_search_table_tp()`, which takes as arguments a source string, a target string, and an initialized search table, and returns a completed search table. The only difference between this function and the earlier `complete_search_table()` should be in the number of candidates passed to `step_distance()`.

*Note: when in the top row or column of the table, you should not consider transposition, because `table[i-2][j-2]` does not exist. Make sure you have an appropriate if statement to deal with this issue!*

In [ ]:
def complete_search_table_tp(source, target, table, edit_cost=rdl_edit_cost):
    # TODO: complete this function
    pass

Run the code cell below to test your function with `">lisp"` as the source and `">tops"` as the target. The tester function converts your table into a pandas DataFrame so that it can be read more easily.

In [ ]:
test(complete_search_table_tp)

## Task 2.8: putting it all together (again) [2 points]

Complete the function `min_edit_distance_tp()`, which is just like `min_edit_distance()`, except it accounts for the possibility of transposition by using `complete_search_table_tp()` instead of `complete_search_table()`.

If your function works, it should tell you that, using the Restricted Damerau-Levenshtein edit cost, the minimum edit distance between `"intention"` and `"execution"` is 5, and the minimum edit distance between `"lisp"` and `"tops"` is 3.

In [ ]:
def min_edit_distance_tp(source, target, edit_cost=rdl_edit_cost):
    # TODO: complete this function
    pass

If your function works, it should tell you that the Restricted Damerau-Levenshtein edit distance between `"lisp"` and `"tops"` is 3. The code cell below will check this for you.

In [ ]:
test(min_edit_distance_tp)

Once you have a working minimum edit distance algorithm, you can swap out the cost function and compute distances under different assumptions. One reason this is useful is because the cost function can be generated by an error model, by taking the cost of an edit to be the negative log-probability of that edit. The minimum edit distance algorithm with this cost function can then be used to find the most likely sequence of errors that transforms a source string into a target string, together with the probability of this transformation.

# Part 3: Applications and effects of noisy channel models [25 pts]

In this part of the assignment, you will write brief responses to questions which direct you to think about applications and effects of language models. You will choose whether your response is based on language technologies (Option A) or language-based research (Option B). **You should only answer questions for a single option; you will not gain extra credit for answering questions for both options. Make sure that you write your chosen option in the designated place at the top of this notebook before you submit the assignment.** 

You can only choose each option *twice* over the four assignments; before choosing an option, please make sure you have not already chosen it for two other assignments.

I will not be grading the quality of your writing, only the quality of your ideas.

Regardless of which option you choose, **make sure that you address all parts of each question**. Not answering questions fully is the most common cause of missed points.

Your responses do not need to be lengthy; make them only as long as is necessary to convey your ideas. Here are some guidelines:  
* *Identify*: 1-2 sentences; give a concrete example of *what* you are identifying  
* *Describe*: 1-3 sentences; give specific details about *what* you are describing  
* *Explain*: 2-4 sentences; give specific details about *how* something works or *why* a statement holds  

When developing your responses, you may find it useful to consult resources such as scholarly publications, news articles, blog posts, etc. (though you are not required to). If you do so, please mention the resources that you consulted, and provide a way in which it can be located (e.g. citation details, a URL, etc.).

## Option A: Real-world effects of noisy channel models

If you choose this option, you will write brief responses to questions which direct you to consider the potential technological applications of noisy channel models, as well as positive and negative effects these applications can have in the real world. You can consider this a purely hypothetical exercise; you are not required to know whether or how a specific technological application actually uses noisy channel models.  
*NB: you should not focus on edit distance or the Viterbi algorithm in this part*

1. Identify a potential technological application of noisy channel models. **[3 points]**  

One potential technological application is speech recognition. As we discussed in class, speech recognition systems convert spoken language into text. It has been adopted in Zoom and graduation ceremonies to make it easier for the audience to engage with the speech.

2. Identify a positive effect that this technological application could have in the real world, and explain why it is a positive effect. **[4 points]**  

A positive effect of speech recognition is improved accessibility for individuals with disabilities, particularly those with mobility impairments or visual disabilities. It can also help people who have trouble with following fast speech.

3. Explain how noisy channel models contribute to the positive effect. **[4 points]** 

Noisy channel models enhance speech recognition by effectively handling the uncertainties inherent in speech recognition, such as variations in accents, background noise, and individual speech patterns. These models can better interpret the intended message despite errors in the transmitted audio.

4. Describe a potential limitation of the use of noisy channel models in creating this positive effect. **[3 points]**  

A potential limitation is that ASR systems may not perform equally well for all users, particularly for those with strong accents or atypical speech patterns. 

5. Identify a negative effect that this technological application could have in the real world, and explain why it is a negative effect. **[4 points]**  

A negative effect of ASR systems is the potential for miscommunication and inaccurate transcriptions, which can lead to misunderstandings in critical situations. This can be especially problematic if it is related to health or social issues.

6. Explain how noisy channel models contribute to the negative effect. **[4 points]**  

Noisy channel models can misinterpret noisy or ambiguous input due to limitations in their training data and inherent noise in the communication channel. The models might generate incorrect hypotheses about the intended message.

7. Describe a way in which the use of noisy channel models could be improved, to mitigate this negative effect. **[3 points]**  

One way to improve the use of noisy channel models is to incorporate diverse and representative training datasets that include a wide range of accents, dialects, and speech variations. 